In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import glob
import seaborn as sns  # for heatmaps
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import sklearn
%matplotlib inline
import glob

# Pending

In [ ]:


def cross_validation(item_func,X,y,n_splits):

    acc_score = []

    kf = KFold(n_splits)
    
    for train_index , test_index in kf.split(X):
        X_train , X_test = X[train_index], X[test_index]
        y_train , y_test = y[train_index], y[test_index]
        train_loader, test_loader = create_dataloader(X_train, X_test, y_train, y_test)
        
        # create model 
        model,optimizer,criterion = item_func()
        
        # train model
        model.fit(X_train,y_train)
        pred_values = model.predict(X_test)
        
        # evaluate
        acc = accuracy_score(pred_values , y_test)
        acc_score.append(acc)

    avg_acc_score = sum(acc_score)/k

    print('accuracy of each fold - {}'.format(acc_score))
    print('Avg accuracy : {}'.format(avg_acc_score))
    
    return 

In [56]:
def contrastive_loss(p1,p2,t=1):
    val = p1.dot(p2.transpose())/t
    val = np.exp(val)
    val = -np.log(np.diag(val)/np.sum(val,axis=1))
    return val

p1 = np.array([
 [ 0.70374682, -0.18682394, -0.68544673],
 [ 0.15465702,  0.32303224,  0.93366556],
 [ 0.53043332, -0.83523217, -0.14500935],
 [ 0.68285685, -0.73054075,  0.00409143],
 [ 0.76652431,  0.61500886,  0.18494479]])

p2 = p1 + 0.1*np.random.random(p1.shape)

# contrastive_loss(p1,p2,0.1)



In [86]:


def contrastive_loss(z1,z2,t=1):
    """
    One to N
    """
    a = torch.mm(z1,z2.transpose(0,1))
    b = torch.norm(z1,dim=1)*torch.norm(z2,dim=1)
    sim_mat = torch.div(a,b)
    mask = torch.ones_like(sim_mat,dtype=bool).fill_diagonal_(0)
    pos = torch.diag(sim_mat).reshape(-1,1)
    neg = sim_mat[mask].reshape(pos.shape[0],-1)
    logits = torch.cat((pos, neg), dim=1)
    labels = torch.zeros(pos.shape[0]).long()
    loss = torch.nn.CrossEntropyLoss()(logits,labels)
    return loss

z1 = torch.tensor(p1)
z2 = torch.tensor(p2)
z3 = torch.zeros_like(z1)

In [ ]:
class Attention(nn.Module):
    def __init__(self,feature_size):
        super(Attention, self).__init__()
        self.linear = nn.Linear(feature_size,1) 

    def forward(self,X):
        assert len(X.shape) == 3
        a = self.linear(X)
        a = torch.relu(a)
        a = F.softmax(a,dim=1)
        return a*X

In [10]:
def intersect(d):
    d = np.array(d)
    ls = d[0]
    for new in d[1:]:
        ls = np.intersect1d(ls,new).tolist()
    return ls


def import_CsiPwr_data(directory):
    """
    import all spectrogram (in pair) in the directory
    """
    print("Importing Data ",end='')
    data = {'X':{},'y':{}}
    for label in os.listdir(directory):
        pfiles = []
        for modality in os.listdir(directory+'/'+label):
            pfiles.append([f.split('.')[0] for f in os.listdir(directory+'/'+label+'/'+modality)])

        print('>',end='')
        common_files = intersect(pfiles)
        
        
        for modality in os.listdir(directory+'/'+label):
            files = [directory+'/'+label+'/'+modality+'/'+pfilename+'.csv' for pfilename in common_files]
            X = import_spectrograms(files)
            y = np.full(X.shape[0], label)
            X[modality]
        
        # selcting available pairs
        pfiles_csi = [f.split('.')[0] for f in os.listdir(directory+'/'+label+'/'+'csi')]
        pfiles_pwr = [f.split('.')[0] for f in os.listdir(directory+'/'+label+'/'+'pwr')]
        available_pairs = np.intersect1d(pfiles_csi,pfiles_pwr).tolist()
        files_csi = [directory+'/'+label+'/'+'csi'+'/'+pfilename+'.csv' for pfilename in available_pairs]
        files_pwr = [directory+'/'+label+'/'+'pwr'+'/'+pfilename+'.csv' for pfilename in available_pairs]
        # importing
        X1 = import_spectrograms(files_csi)
        X2 = import_spectrograms(files_pwr)
        y = np.full(X1.shape[0], label)
        assert X1.shape[0] == X2.shape[0]
        data['X1'].append(X1)
        data['X2'].append(X2)
        data['y'].append(y)
    print(" Complete")
    return np.concatenate(data['X1']), np.concatenate(data['X2']), np.concatenate(data['y'])

In [24]:
from data.transformation import label_encode

In [43]:
arr = np.array(['car','cat','cat','cow','cow','cow'])
arr,lb = label_encode(arr)
class_weight = torch.FloatTensor([1-w for w in pd.Series(arr).value_counts(normalize=True).sort_index().tolist()])

In [44]:
class_weight

tensor([0.8333, 0.6667, 0.5000])

In [38]:
lb.classes_

array(['car', 'cat', 'cow'], dtype='<U3')

# Test code

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchsummary import summary

In [23]:
np.random.seed(1024)
torch.manual_seed(1024)


NUM_EPOCHS = 100
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = DEVICE
PATH = '.'
NUM_WORKERS = 0

torch.cuda.set_device(DEVICE)

In [4]:
DIRC = 'E:/external_data/Experiment4/Spectrogram_data_csv_files/CSI_data'

In [5]:
from data import prepare_double_source,prepare_single_source

In [ ]:
_ , train_loader, test_loader, lb, weight = prepare_double_source(directory=DIRC,
                                                          modality='single',
                                                          axis=1,
                                                          train_size=0.8,
                                                          joint='joint',
                                                          p=None,
                                                          sampling='weight',
                                                          batch_size=64,
                                                          num_workers=NUM_WORKERS)

In [6]:
train_loader,test_loader,lb,class_weight = prepare_single_source(directory=DIRC,axis=3,train_size=0.8,sampling='weight',batch_size=64,num_workers=0)

Importing Data >>>>>> Complete


2021-02-23 14:39:21,574 - [INFO] - NumExpr defaulting to 8 threads.


X_train:  (760, 1, 65, 501) 	y_train:  (760,) 	X_test:  (190, 1, 65, 501) 	y_test:  (190,)
class:  ['lay' 'picking' 'sit' 'stand_s' 'walking' 'waving']
class_size:  tensor([0.0526, 0.1303, 0.1197, 0.1408, 0.3855, 0.1711])


In [102]:
from models.utils import Lambda,Classifier,ED_module

class Encoder(nn.Module):
    """
    Three layer Encoder for spectrogram (1,65,65), 3 layer
    """
    def __init__(self,num_filters):
        super(Encoder, self).__init__()
        l1,l2,l3 = num_filters
        ### 1st ###
        self.conv1 = nn.Conv2d(1,l1,kernel_size=5,stride=1)
        self.norm1 = nn.BatchNorm2d(l1) # nn.BatchNorm2d()
        self.actv1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=(2,2))
        ### 2nd ###
        self.conv2 = nn.Conv2d(l1,l2,kernel_size=4,stride=2)
        self.norm2 = nn.BatchNorm2d(l2)
        self.actv2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=(2,2))
        ### 3rd ###
        self.conv3 = nn.Conv2d(l2,l3,kernel_size=3,stride=3)
        self.norm3 = Lambda(lambda x:x)
        self.actv3 = nn.Tanh()
        self.pool3 = nn.MaxPool2d(kernel_size=(2,2))

    def forward(self,X):
        X = self.pool1(self.actv1(self.norm1(self.conv1(X))))
        X = self.pool2(self.actv2(self.norm2(self.conv2(X))))
        X = self.pool3(self.actv3(self.norm3(self.conv3(X))))
        X = torch.flatten(X, 1)
        # print(X.shape)
        return X

    
def create_model():
    out = 128
    enc = Encoder([128,128,out]) # 1440
    # summary(enc,(1,65,501),batch_size=64)
    dec = Classifier(10*out,128,6)
    model = ED_module(enc,dec)
    return model

model = create_model().to(DEVICE)
summary(model,(1,65,501),batch_size=64)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [64, 128, 61, 497]           3,328
       BatchNorm2d-2         [64, 128, 61, 497]             256
              ReLU-3         [64, 128, 61, 497]               0
         MaxPool2d-4         [64, 128, 30, 248]               0
            Conv2d-5         [64, 128, 14, 123]         262,272
       BatchNorm2d-6         [64, 128, 14, 123]             256
              ReLU-7         [64, 128, 14, 123]               0
         MaxPool2d-8           [64, 128, 7, 61]               0
            Conv2d-9           [64, 128, 2, 20]         147,584
           Lambda-10           [64, 128, 2, 20]               0
             Tanh-11           [64, 128, 2, 20]               0
        MaxPool2d-12           [64, 128, 1, 10]               0
          Encoder-13                 [64, 1280]               0
           Linear-14                  [

In [103]:
criterion = nn.CrossEntropyLoss(weight=class_weight).to(DEVICE)
optimizer = torch.optim.Adam(list(model.parameters()), lr=0.0005)

In [104]:
from train import train

model, record = train(model=model,
                      train_loader=train_loader,
                      criterion=criterion,
                      optimizer=optimizer,
                      end=NUM_EPOCHS,
                      start = 1,
                      test_loader = test_loader,
                      device = DEVICE,
                      regularize = True)

Start Training
Epoch 1: >

RuntimeError: CUDA out of memory. Tried to allocate 234.00 MiB (GPU 0; 4.00 GiB total capacity; 2.79 GiB already allocated; 12.84 MiB free; 2.81 GiB reserved in total by PyTorch)

In [50]:
from train import evaluation,record_log,save

In [96]:
cmtx,cls = evaluation(model,test_loader,label_encoder=lb)

               0         1          2          3          4          5  \
precision  0.750   0.62069   0.500000   0.714286   0.863014   0.860465   
recall     0.375   0.62069   0.526316   0.555556   0.954545   0.902439   
f1-score   0.500   0.62069   0.512821   0.625000   0.906475   0.880952   
support    8.000  29.00000  19.000000  27.000000  66.000000  41.000000   

           accuracy   macro avg  weighted avg  
precision  0.768421    0.718076      0.763283  
recall     0.768421    0.655758      0.768421  
f1-score   0.768421    0.674323      0.760868  
support    0.768421  190.000000    190.000000  


In [88]:
MAIN_NAME = 'Trainmode_normal_Network_shallowv2_Data_exp4nuc1'
record_log(MAIN_NAME,NUM_EPOCHS,record,cmtx=cmtx,cls=cls)
save(MAIN_NAME,model,optimizer,NUM_EPOCHS)

save checkpoint in : ./models/saved_models/Trainmode_normal_Network_shallowv2_Data_exp4nuc1_checkpoint_100__2021_02_23_15_28


In [101]:
del model
torch.cuda.empty_cache()